In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import autokeras as ak
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------------
# 1. Preparación de los Datos
# -------------------------------

# Directorio principal de imágenes
images_dir = 'arcgis-survey-images'

# Verificar si el directorio existe
if not os.path.isdir(images_dir):
    raise FileNotFoundError(f"El directorio '{images_dir}' no existe. Por favor, verifica la ruta.")

# Obtener las clases a partir de los nombres de los subdirectorios
class_names = sorted([d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))])
print(f"Clases encontradas: {class_names}")

# Recopilar rutas de imágenes y etiquetas
data = []
for class_label in class_names:
    class_dir = os.path.join(images_dir, class_label)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(class_dir, img_name)
            data.append({'image': img_path, 'label': class_label})

# Crear un DataFrame
df = pd.DataFrame(data)
print(f"Total de imágenes: {len(df)}")

# Dividir en conjuntos de entrenamiento y validación
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=123)
print(f"Entrenamiento: {len(train_df)} imágenes")
print(f"Validación: {len(valid_df)} imágenes")

# Guardar los DataFrames en archivos CSV (opcional)
train_df.to_csv('train_data.csv', index=False)
valid_df.to_csv('valid_data.csv', index=False)

# -------------------------------
# 2. Preprocesamiento Avanzado de Imágenes
# -------------------------------

# Definir las transformaciones de preprocesamiento
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    
    transform = A.Compose([
        A.Resize(128, 128),
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),
        A.GaussianBlur(blur_limit=(3,7), p=0.2),
        A.Canny(p=0.1),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    
    transformed = transform(image=image)
    return transformed['image']

# Función para cargar y preprocesar imágenes
def load_and_preprocess_image(path, label):
    # Convertir la ruta de la imagen a un tensor
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.uint8)
    
    # Convertir a numpy para usar Albumentations
    image = tf.numpy_function(preprocess_image, [path], tf.float32)
    
    return image, label

# Crear conjuntos de datos de TensorFlow
train_ds = tf.data.Dataset.from_tensor_slices((train_df['image'].values, train_df['label'].values))
valid_ds = tf.data.Dataset.from_tensor_slices((valid_df['image'].values, valid_df['label'].values))

# Mapeo de etiquetas a índices
label_to_index = {label: index for index, label in enumerate(class_names)}
train_ds = train_ds.map(lambda x, y: (x, label_to_index[y]))
valid_ds = valid_ds.map(lambda x, y: (x, label_to_index[y]))

# Aplicar el preprocesamiento
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batching y Prefetching
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# -------------------------------
# 3. Implementación de AutoML con AutoKeras
# -------------------------------

# Definir el ImageClassifier de AutoKeras
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=10,  # Número de modelos a probar
    metrics=['accuracy']
)

# Entrenar el modelo
print("Iniciando el entrenamiento con AutoKeras...")
clf.fit(
    train_ds,
    epochs=50,
    validation_data=valid_ds
)

# Evaluar el modelo
loss, accuracy = clf.evaluate(valid_ds)
print(f'Precisión en el conjunto de validación: {accuracy:.4f}')

# -------------------------------
# 4. Evaluación del Modelo
# -------------------------------

# Obtener predicciones en el conjunto de validación
# Para obtener etiquetas verdaderas y predichas
def get_labels_and_predictions(model, dataset):
    y_true = []
    y_pred = []
    for batch in dataset:
        images, labels = batch
        preds = model.predict(images)
        preds = tf.argmax(preds, axis=1).numpy()
        y_true.extend(labels.numpy())
        y_pred.extend(preds)
    return y_true, y_pred

y_true, y_pred = get_labels_and_predictions(clf, valid_ds)

# Mapear índices a etiquetas
index_to_label = {index: label for label, index in label_to_index.items()}
y_true_labels = [index_to_label[idx] for idx in y_true]
y_pred_labels = [index_to_label[idx] for idx in y_pred]

# Generar el reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true_labels, y_pred_labels, target_names=class_names))

# Crear la matriz de confusión
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta')
plt.title('Matriz de Confusión')
plt.show()

# -------------------------------
# 5. Visualización de Predicciones
# -------------------------------

# Mostrar algunas imágenes con sus predicciones
import matplotlib.pyplot as plt
import random

def show_predictions(model, df, num_images=5):
    samples = df.sample(n=num_images, random_state=42)
    for idx, row in samples.iterrows():
        img_path = row['image']
        true_label = row['label']
        image = Image.open(img_path).convert('RGB')
        image_np = np.array(image)
        image_np = preprocess_image(img_path).transpose(1, 2, 0)
        image_np = (image_np * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406])  # Desnormalizar
        image_np = np.clip(image_np, 0, 1)
        
        # Expandir dimensiones para predecir
        image_input = np.expand_dims(preprocess_image(img_path), axis=0)
        pred = model.predict(image_input)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_label = index_to_label[pred_class]
        
        plt.figure(figsize=(4,4))
        plt.imshow(image_np)
        plt.title(f"Verdadero: {true_label}\nPredicción: {pred_label}")
        plt.axis('off')
        plt.show()

# Mostrar 5 predicciones aleatorias
show_predictions(clf, valid_df, num_images=5)

# -------------------------------
# 6. Guardar y Cargar el Modelo
# -------------------------------

# Guardar el modelo
clf.save("autokeras_plague_classifier")
print("Modelo guardado en 'autokeras_plague_classifier'.")

# Cargar el modelo guardado (opcional)
loaded_clf = ak.ImageClassifier.load("autokeras_plague_classifier")
print("Modelo cargado desde 'autokeras_plague_classifier'.")

# Evaluar nuevamente para verificar
loss, accuracy = loaded_clf.evaluate(valid_ds)
print(f'Precisión del modelo cargado en el conjunto de validación: {accuracy:.4f}')


ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'